<a href="https://colab.research.google.com/github/jw9603/AIM5021_41-/blob/main/11thweek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. RNN을 이용하여 텍스트 생성하기

In [1]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

True

### 1. 데이터에 대한 이해와 전처리

In [2]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [3]:
text = """경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

단어 집합을 생성하고 크기를 확인해보자. 단어 집합의 크기를 저장할 때는 케라스 토크나이저의 정수 인코딩은 인덱스가 1부터 시작하지만, 패딩을 위한 0을 고려하여 +1을 해준다.

In [4]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
vocab_size = len(tokenizer.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 12


각 단어와 단어에 부여된 정수 인덱스를 출력해보자

In [5]:
print(tokenizer.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


훈련 데이터를 만들어보자!

In [7]:
sequences = list()
for line in text.split('\n'): # 줄바꿈 문자를 기준으로 문장 토큰화
    encoded = tokenizer.texts_to_sequences([line])[0]
    print(encoded)
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수: %d' % len(sequences))

[2, 3, 1, 4, 5]
[]
[6, 1, 7]
[]
[8, 1, 9, 10, 1, 11]
[]
학습에 사용할 샘플의 개수: 11


In [8]:
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


위의 데이터는 아직 레이블로 사용될 단어를 분리하지 않은 훈련 데이터입니다. [2, 3]은 [경마장에, 있는]에 해당되며 [2, 3, 1]은 [경마장에, 있는, 말이]에 해당됩니다. 전체 훈련 데이터에 대해서 맨 우측에 있는 단어에 대해서만 레이블로 분리해야 합니다.

우선 전체 샘플에 대해서 길이를 일치시켜 줍니다. 가장 긴 샘플의 길이를 기준으로 합니다. 현재 육안으로 봤을 때, 길이가 가장 긴 샘플은 [8, 1, 9, 10, 1, 11]이고 길이는 6입니다. 이를 코드로는 다음과 같이 구할 수 있습니다.

In [9]:
max_len = max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 6


전체 훈련 데이터에서 가장 긴 샘플의 길이가 6임을 확인하였다. 전체 샘플의 길이를 6으로 패딩한다.

In [10]:
sequences = pad_sequences(sequences, maxlen=max_len,padding='pre')

In [11]:
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


길이가 6보다 짧은 모든 샘플에 대해서 앞에 0을 채워서 모든 샘플의 길이를 6으로 바꿨습니다. 이제 각 샘플의 마지막 단어를 레이블로 분리합시다. 레이블의 분리는 Numpy를 이용해서 가능합니다. 리스트의 마지막 값을 제외하고 저장한 것은 X, 리스트의 마지막 값만 저장한 것은 y. 이는 레이블에 해당됩니다.

In [12]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [13]:
print(X)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]


In [14]:
print(y)

[ 3  1  4  5  1  7  1  9 10  1 11]


레이블이 분리되었다. RNN 모델에 훈련 데이터를 훈련 시키기 전에 레이블에 대해서 원-핫 인코딩을 수행한다.

In [15]:
y = to_categorical(y,num_classes=vocab_size)

In [16]:
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


### 2. 모델 설계하기

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN, LSTM

In [26]:
embedding_dim = 10
hidden_units = 128

model1 = Sequential()
model1.add(Embedding(vocab_size, embedding_dim))
model1.add(LSTM(hidden_units))
model1.add(Dense(vocab_size, activation='softmax'))
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model1.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
1/1 - 2s - loss: 2.4858 - accuracy: 0.0000e+00 - 2s/epoch - 2s/step
Epoch 2/200
1/1 - 0s - loss: 2.4799 - accuracy: 0.4545 - 11ms/epoch - 11ms/step
Epoch 3/200
1/1 - 0s - loss: 2.4741 - accuracy: 0.3636 - 9ms/epoch - 9ms/step
Epoch 4/200
1/1 - 0s - loss: 2.4680 - accuracy: 0.3636 - 9ms/epoch - 9ms/step
Epoch 5/200
1/1 - 0s - loss: 2.4615 - accuracy: 0.3636 - 9ms/epoch - 9ms/step
Epoch 6/200
1/1 - 0s - loss: 2.4546 - accuracy: 0.3636 - 10ms/epoch - 10ms/step
Epoch 7/200
1/1 - 0s - loss: 2.4471 - accuracy: 0.3636 - 9ms/epoch - 9ms/step
Epoch 8/200
1/1 - 0s - loss: 2.4388 - accuracy: 0.3636 - 10ms/epoch - 10ms/step
Epoch 9/200
1/1 - 0s - loss: 2.4296 - accuracy: 0.3636 - 13ms/epoch - 13ms/step
Epoch 10/200
1/1 - 0s - loss: 2.4194 - accuracy: 0.3636 - 11ms/epoch - 11ms/step
Epoch 11/200
1/1 - 0s - loss: 2.4080 - accuracy: 0.3636 - 10ms/epoch - 10ms/step
Epoch 12/200
1/1 - 0s - loss: 2.3951 - accuracy: 0.3636 - 11ms/epoch - 11ms/step
Epoch 13/200
1/1 - 0s - loss: 2.3807 - accura

In [18]:
embedding_dim = 10
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
1/1 - 1s - loss: 2.4675 - accuracy: 0.1818 - 1s/epoch - 1s/step
Epoch 2/200
1/1 - 0s - loss: 2.4555 - accuracy: 0.3636 - 10ms/epoch - 10ms/step
Epoch 3/200
1/1 - 0s - loss: 2.4437 - accuracy: 0.4545 - 8ms/epoch - 8ms/step
Epoch 4/200
1/1 - 0s - loss: 2.4321 - accuracy: 0.5455 - 9ms/epoch - 9ms/step
Epoch 5/200
1/1 - 0s - loss: 2.4205 - accuracy: 0.5455 - 9ms/epoch - 9ms/step
Epoch 6/200
1/1 - 0s - loss: 2.4089 - accuracy: 0.6364 - 9ms/epoch - 9ms/step
Epoch 7/200
1/1 - 0s - loss: 2.3972 - accuracy: 0.5455 - 7ms/epoch - 7ms/step
Epoch 8/200
1/1 - 0s - loss: 2.3853 - accuracy: 0.5455 - 6ms/epoch - 6ms/step
Epoch 9/200
1/1 - 0s - loss: 2.3731 - accuracy: 0.5455 - 8ms/epoch - 8ms/step
Epoch 10/200
1/1 - 0s - loss: 2.3606 - accuracy: 0.5455 - 7ms/epoch - 7ms/step
Epoch 11/200
1/1 - 0s - loss: 2.3476 - accuracy: 0.5455 - 8ms/epoch - 8ms/step
Epoch 12/200
1/1 - 0s - loss: 2.3342 - accuracy: 0.5455 - 8ms/epoch - 8ms/step
Epoch 13/200
1/1 - 0s - loss: 2.3201 - accuracy: 0.5455 - 8ms

In [19]:
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        # 현재 단어에 대한 정수 인코딩과 패딩
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=5, padding='pre')
        # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items(): 
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면 break
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

In [20]:
print(sentence_generation(model, tokenizer, '경마장에', 4))

경마장에 있는 말이 뛰고 있다


In [27]:
print(sentence_generation(model1, tokenizer, '경마장에', 4))

경마장에 있는 말이 뛰고 있다


In [21]:
print(sentence_generation(model, tokenizer, '그의', 2))

그의 말이 법이다


In [28]:
print(sentence_generation(model1, tokenizer, '그의', 2))

그의 말이 법이다


In [22]:
print(sentence_generation(model, tokenizer, '가는', 5))

가는 말이 고와야 오는 말이 곱다


In [29]:
print(sentence_generation(model1, tokenizer, '가는', 5))

가는 말이 고와야 오는 말이 곱다


## 1D CNN으로 IMDB 분류하기

### 1. 데이터에 대한 전처리

In [30]:
from tensorflow.keras import datasets
from tensorflow.keras.preprocessing.sequence import pad_sequences

최대 10,000개의 단어만을 허용하여 데이터를 로드한다.

In [32]:
vocab_size = 10000
(X_train, y_train), (X_test, y_test) = datasets.imdb.load_data(num_words=vocab_size)

17464789/17464789 [==============================] - 1s 0us/step


In [33]:
print(X_train[:5])

[list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32])
 list([1, 194, 1153, 194, 8255, 78, 228, 5, 6, 1463, 

각 샘플의 길이가 긴 관계로 출력 시 중간 내용은 중략하였습니다. 각 샘플은 이미 정수 인코딩까지 전처리가 된 상태입니다. 하지만 각 샘플들의 길이는 서로 다르죠? 패딩을 진행하여 모든 샘플들의 길이를 200으로 맞춥니다.

In [34]:
max_len = 200
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

패딩이 되었는지 크기(shape)를 확인해봅시다.

In [35]:
print('X_train의 크기(shape) :',X_train.shape)
print('X_test의 크기(shape) :',X_test.shape)

X_train의 크기(shape) : (25000, 200)
X_test의 크기(shape) : (25000, 200)


훈련 데이터, 테스트 데이터 각 25,000 샘플이 전부 길이 200을 가지는 것을 확인할 수 있습니다. y_train도 출력해봅시다.

In [36]:
print(y_train[:5])

[1 0 0 1 0]


### 2. 1D CNN으로 IMDB 리뷰 분류하기

In [37]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

embedding_dim = 256 # 임베딩 벡터의 차원
dropout_ratio = 0.3 # 드롭아웃 비율
num_filters = 256 # 커널의 수
kernel_size = 3 # 커널의 크기
hidden_units = 128 # 뉴런의 수

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(Dropout(dropout_ratio))
model.add(Conv1D(num_filters, kernel_size, padding='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(hidden_units, activation='relu'))
model.add(Dropout(dropout_ratio))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), callbacks=[es, mc])

Epoch 1/20
782/782 [==============================] - ETA: 0s - loss: 0.4024 - acc: 0.8048
Epoch 1: val_acc improved from -inf to 0.87220, saving model to best_model.h5
782/782 [==============================] - 162s 206ms/step - loss: 0.4024 - acc: 0.8048 - val_loss: 0.2960 - val_acc: 0.8722
Epoch 2/20
782/782 [==============================] - ETA: 0s - loss: 0.2061 - acc: 0.9194
Epoch 2: val_acc improved from 0.87220 to 0.89176, saving model to best_model.h5
782/782 [==============================] - 162s 207ms/step - loss: 0.2061 - acc: 0.9194 - val_loss: 0.2636 - val_acc: 0.8918
Epoch 3/20
782/782 [==============================] - ETA: 0s - loss: 0.0927 - acc: 0.9694
Epoch 3: val_acc did not improve from 0.89176
782/782 [==============================] - 161s 206ms/step - loss: 0.0927 - acc: 0.9694 - val_loss: 0.3407 - val_acc: 0.8826
Epoch 4/20
782/782 [==============================] - ETA: 0s - loss: 0.0419 - acc: 0.9861
Epoch 4: val_acc did not improve from 0.89176
782/782 [=

In [38]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

782/782 [==============================] - 31s 40ms/step - loss: 0.2636 - acc: 0.8918

 테스트 정확도: 0.8918
